In [1]:
from google.cloud import bigquery
import metodos
import json

In [2]:
client = bigquery.Client.from_service_account_json('../bq-gcs-creds.json')

In [3]:
# Abre el archivo JSON en modo de lectura
with open('config -estructura.json', 'r', encoding='utf-8') as f:
    # Carga los datos del archivo JSON en una estructura de datos de Python
    config_json = json.load(f)

# Ahora puedes acceder a los datos en la estructura de datos de Python
print(config_json)

{'proyecto': [{'nombre': 'latam-recovery', 'datasets': [{'nombre': 'atiendo20_02', 'tablas': [{'nombre': 'ivr_marcaron_1', 'columnas': [{'nombre': 'ivm_id', 'tipo': 'INTEGER'}, {'nombre': 'cuil', 'tipo': 'INTEGER'}, {'nombre': 'ivm_telefono', 'tipo': 'INTEGER'}, {'nombre': 'ivm_fecha', 'tipo': 'TIMESTAMP'}, {'nombre': 'ivm_fecha1', 'tipo': 'DATE'}, {'nombre': 'civ_loans_id_campania', 'tipo': 'INTEGER'}, {'nombre': 'uniqueid', 'tipo': 'FLOAT'}]}]}, {'nombre': 'telecom_pre_baja', 'tablas': [{'nombre': 'temp_archivo_MA', 'columnas': [{'nombre': 'OBJ_ID', 'tipo': 'INT64'}, {'nombre': 'CUENTA_CODE', 'tipo': 'INT64'}, {'nombre': 'ID_CLIENTE', 'tipo': 'INT64'}, {'nombre': 'NRO_TRAMITE', 'tipo': 'INT64'}, {'nombre': 'NRO_NOMINA', 'tipo': 'INT64'}, {'nombre': 'FECHA_INICIO_ASIGNACION', 'tipo': 'DATE'}, {'nombre': 'FECHA_FIN_ASIGNACION', 'tipo': 'DATE'}, {'nombre': 'MONTO_DE_DEUDA_ASIGNADO', 'tipo': 'FLOAT64'}, {'nombre': 'GESTOR_ASIGNADO', 'tipo': 'INT64'}, {'nombre': 'NOMBRE_DEUDOR', 'tipo': '

Parametros

In [20]:
# Especifica los detalles del proyecto, el conjunto de datos y la tabla que deseas borrar o crear

#project_id = 'latam-recovery' #'nombre-del-proyecto'
dataset_id = 'telecom' # 'nombre-del-conjunto-de-datos'
table_id = 'cartera_extraPay' # 'nombre-de-la-tabla'
#C:\Users\Usuario\Proyectos_Anaconda\WORK LR\ISLA 1 UALA PAGOS
# Archivo csv
file = 'C:/Users/Usuario/Proyectos_Anaconda/WORK LR/Telecom/Carteras/Base Extra Pay 2024_02_29_nuevo'
tipo = 'csv'

In [21]:
# Buscaqueda del conjunto de datos y la tabla para 
# Definir el esquema de la tabla. Esto es opcional

datasets = config_json['proyecto'][0]['datasets']
schema = list()
dataset_encontrado = False
tabla_encontrado = False

for dataset in datasets:
    if dataset['nombre'] == dataset_id:
        
        dataset_encontrado = True
        print(f'Se encontro el dataset: {dataset_id}')

        for tabla in dataset['tablas']:
            if tabla['nombre2'] == table_id:
                
                tabla_encontrado = True
                print(f'Se encontro la tabla: {table_id}')
                
                for columna in tabla['columnas2']:
                    schema.append(bigquery.SchemaField(columna['nombre'], columna['tipo']))

if not dataset_encontrado:
    print("No se encontró el dataset")

if not tabla_encontrado:
    print("No se encontró la tabla")

Se encontro el dataset: telecom
Se encontro la tabla: cartera_extraPay


In [22]:
schema

[SchemaField('Apellido_de_Cliente', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Nombre_de_Cliente', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Tipo_DNI', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Numero_DNI', 'INT64', 'NULLABLE', None, None, (), None),
 SchemaField('Edad', 'INT64', 'NULLABLE', None, None, (), None),
 SchemaField('Telefono_de_Contacto', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Mail', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Actividad__Profesion_', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Saldo_Capital', 'FLOAT64', 'NULLABLE', None, None, (), None),
 SchemaField('Intereses', 'FLOAT64', 'NULLABLE', None, None, (), None),
 SchemaField('Saldo_Total_Consolidado', 'FLOAT64', 'NULLABLE', None, None, (), None),
 SchemaField('Dias_de_Mora', 'INT64', 'NULLABLE', None, None, (), None)]

Borrar una tabla

In [23]:
# Llama a la función para borrar la tabla
metodos.borrar_tabla_bigquery(client, dataset_id, table_id)

Crear una tabla

In [24]:
# Crea una referencia a la tabla de destino:
table_ref = client.dataset(dataset_id).table(table_id)

In [25]:
# Crea una configuración de carga para especificar los detalles del archivo CSV
job_config = bigquery.LoadJobConfig(
    schema=schema,
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,

    #autodetect=False,
    #field_delimiter=';'
    #quote_character='"',
    #allow_quoted_newlines=True,
    #time_partitioning=None,
    #destination_schema_update_options=[],
    #write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

# Asumimos que el archivo CSV tiene una fila de encabezado y que los datos reales comienzan desde la segunda fila. 
# Ajustar skip_leading_rows si el archivo CSV no sigue esta estructura.

In [26]:
with open(f'{file}.{tipo}', 'rb') as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()

LoadJob<project=latam-recovery, location=us-central1, id=5d3a3cbe-8578-429c-a2be-c03d281a7f05>

In [27]:
# Consulta la tabla para verificar que se haya convertido la columna de fecha a TIMESTAMP
table = client.get_table(table_ref)
print(table.schema)

[SchemaField('Apellido_de_Cliente', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Nombre_de_Cliente', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Tipo_DNI', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Numero_DNI', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('Edad', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('Telefono_de_Contacto', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Mail', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Actividad__Profesion_', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('Saldo_Capital', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('Intereses', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('Saldo_Total_Consolidado', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('Dias_de_Mora', 'INTEGER', 'NULLABLE', None, None, (), None)]
